In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from PIL import Image
import matplotlib.pyplot as plt
pd.plotting.register_matplotlib_converters()
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS

import seaborn as sns# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#reading date from CSV file
csv_file ='/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv'
df = pd.read_csv(csv_file,skiprows=[1])
df.head()

# Big View
Who are take this survey? and what is programming they do?

- mostly youth from 22-39

In [ ]:
#Age
fig = px.histogram(df, x="Q1",
        marginal="box",
        ).update_xaxes(categoryorder="category ascending",)
fig.show()

# Gender Looking Over

In [ ]:
#Gender plotly Pie
gender = df['Q2'].value_counts()
fig = px.pie(values=gender.values,names=gender.index)
fig.update_traces(textposition='inside',textinfo='percent+label')
fig.show()

## Countries 

In [ ]:
#plotly Chorophleth map
country = df['Q3'].value_counts().reset_index()
fig = px.choropleth(country, locations="index",
                    locationmode = 'country names',
                    color="Q3", # lifeExp is a column of gapminder
                    hover_name="index", # column to add to hover information
                     
                    color_continuous_scale=px.colors.sequential.Blugrn)

fig.show()

# Education

In [ ]:
#
rst = df['Q4'].value_counts()
fig = px.pie(values= rst.values,names=rst.index,labels=False)
fig.update_traces(textposition='inside',textinfo='percent+label',showlegend=False)
fig

# Programming Languages 
**Python** in top of Most using With Programmers.

In [ ]:
#plotly HBar
lang = df.filter(regex='Q7').copy()
lang.set_axis(['Python','R','SQL','C','C++','Java',
                'Javascript','Julia','Swift','Bash',
                'MATLAB','None','Other'],
                axis=1,inplace=True)
lang.fillna(0,inplace=True)
lang = lang.apply(lambda x: [1 if n else 0 for n in x ])
sorted_lang = lang.sum().sort_values(ascending=True)
fig = px.bar(sorted_lang, text='value', labels={'value':'Use on a regular',
                                                'index':'programming languages'},orientation='h',)
fig.show()

## What is programming languages mostly programmer know together.
- corrolation  programming languages with it self use on regular



In [ ]:
#Plotly heatmap 
# corrolation Q7 with it self programming languages use on regular
fig = px.imshow(lang.corr(),color_continuous_scale='purples')
fig.show()

## Which is the best programing languages programmers advice to learn in the first programing language?
**Python** without competitor

In [ ]:
#plotly HBar
rst = df['Q8'].value_counts()
fig = px.pie(v alues= rst.values,names=rst.index,hole=0.9)
fig.update_traces(textposition='inside',textinfo='percent+label')
fig.show()

# popular Data Visual Liberary

In [ ]:
#Data Visual Libs
visual = df.filter(regex='Q14').copy()
visual.set_axis(['Matplotlib','Seaborn','Plotly / Plotly Express',
                'Ggplot / ggplot2','Shiny','D3.js','Altair',
                'Bokeh','Geoplotlib','Leaflet / Folium','None','Other'],
                axis=1,inplace=True)
visual.fillna(0,inplace=True)
visual = visual.apply(lambda x: [1 if n else 0 for n in x ])
sorted_visual = visual.sum()#.sort_values(ascending=True)
fig = px.bar(sorted_visual, text='value', labels={'value':'Use on a regular',
                                                'index':'data visualization libraries'})#,orientation='h',)
fig.show()

# What is regural ML framwork use?

In [ ]:

ml = df.filter(regex='Q16').copy()
ml.set_axis(['Scikit-learn','TensorFlow','Keras','PyTorch',
                'Fast.ai','MXNet','Xgboost','LightGBM','CatBoost',
                'Prophet','H2O 3','Caret','Tidymodels',
                'JAX','None','Other'],
                axis=1,inplace=True)
ml.fillna(0,inplace=True)
ml = ml.apply(lambda x: [1 if n else 0 for n in x ])
sorted_ml = ml.sum()#.sort_values(ascending=True)
fig = px.bar(sorted_ml, text='value', labels={'value':'Use on a regular',
                                                'index':'machine learning frameworks'})
fig.show()

# Age in Details
**Woman** in 22-24 in peak of it's own line,but **Men** go in peak in 25-29

In [ ]:
#Gender destribution with age groups
age_group = df.groupby(['Q1','Q2'])['Q3'].count().reset_index()
fig= px.line(age_group ,x='Q1',y ='Q3',color='Q2')
fig.show()

## age/language programming

In [ ]:
age = df.filter(regex='Q(7|1$)').copy()
age.set_axis(['age']+col_lang,axis=1,inplace=True)
age = age.groupby('age').count()

fig = px.imshow(age,labels={'x':'Programming Language','y':'Age Group'},color_continuous_scale='purples')
fig.show()

# Title Rule in details

## How many years title have been writing code/programing?

In [ ]:
coding_years = df[['Q3','Q5','Q6']].groupby(by=['Q5','Q6']).count().unstack()
coding_years = coding_years.droplevel(0,axis=1)

fig = px.imshow(coding_years,labels={'y':'Title Ruls','x':'Age Group'},
                                color_continuous_scale='oranges',)
fig.update_xaxes(side="top")
fig.show()


# Countries in details

## Country/hosted notebook

In [ ]:

coding_years = df.filter(regex='Q(10|3$)').copy()
coding_years = coding_years.groupby('Q3').count()
coding_years.set_axis(['Kaggle Notebooks','Colab Notebooks','Azure Notebooks',
                'Paperspace / Gradient','Binder / JupyterHub','Code Ocean',
                'IBM Watson Studio','Amazon Sagemaker Studio','Amazon EMR Notebooks',
                'Google Cloud AI Platform Notebooks','Google Cloud Datalab Notebooks',
                'Databricks Collaborative Notebooks','None','Other'],
                axis=1,inplace=True)

fig = px.area(coding_years,labels={'y':'Title Ruls','x':'Age Group'})
fig.show()
